## Test accuracy on 20 CIFAR-100 classes using Vgg16

### Import all the necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

2025-05-26 23:16:42.389773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748279802.409701   29203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748279802.415229   29203 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748279802.429519   29203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748279802.429538   29203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748279802.429540   29203 computation_placer.cc:177] computation placer alr

###  Load CIFAR-100 fine labels (100-class)

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [3]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [4]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 224x224

In [5]:
x_train_20 = tf.image.resize(x_train_20, (224, 224)).numpy()
x_test_20 = tf.image.resize(x_test_20, (224, 224)).numpy()

I0000 00:00:1748279807.795575   29203 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 58 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-05-26 23:16:57.832203: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.0KiB (rounded to 1280)requested by op ScratchBuffer
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-26 23:16:57.832225: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-26 23:16:57.832234: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-26 23:16:57.83

FailedPreconditionError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} Failed to allocate scratch buffer for device 0

### Preprocess

In [ ]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [ ]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Use Vgg16 + Custom Head

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.Flatten(),  
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 9. Train model
model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

# 10. Evaluate
loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using VGG16:", acc)

2025-05-24 16:24:38.882148: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5419008000 exceeds 10% of free system memory.


Epoch 1/5


2025-05-24 16:25:12.309519: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2025-05-24 16:25:12.496529: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.


141/141 [==============================] - 962s 7s/step - loss: 2.9414 - accuracy: 0.4007 - val_loss: 1.1951 - val_accuracy: 0.6660
Epoch 2/5
141/141 [==============================] - 956s 7s/step - loss: 1.4908 - accuracy: 0.5688 - val_loss: 1.0875 - val_accuracy: 0.6830
Epoch 3/5
141/141 [==============================] - 957s 7s/step - loss: 1.3249 - accuracy: 0.6164 - val_loss: 1.0489 - val_accuracy: 0.7110
Epoch 4/5
141/141 [==============================] - 962s 7s/step - loss: 1.1433 - accuracy: 0.6553 - val_loss: 1.0243 - val_accuracy: 0.7490
Epoch 5/5
63/63 [==============================] - 192s 3s/step - loss: 0.9046 - accuracy: 0.7590
Test accuracy on 20 CIFAR-100 classes using VGG16: 0.7590000033378601
